# What is this notebook about?
Although the datasets used in this project are not that big, do you:
- Have a machine that cannot handle loading all the data at once due to limited RAM?
- Not have access to your own GPU with CUDA to run training?

If this is the case, don't worry! In this notebook, I will show you how to leverage Kaggle's free computing resource to run training!

**Prerequisites:**
- All you need is a Kaggle account!
- And maybe VSCode too if you are planning to edit the
    notebook locally while using Kaggle's kernel.

# Setting Things Up
In this section, we will:
- Package and upload the necessary source code to Kaggle as a dataset.
- Upload the training script as a Utility Script on Kaggle.
- Create a notebook to assemble the actual datasets, source code and training script.

## Let's Start Locally
As you can see, this project includes quite a lot of pacakged code that can not be copied straight into a notebook to be run on Kaggle. So we must package these code and upload to Kaggle first so we can reference them later on.

From your terminal of choice, navigate to the `src` directory:
```bash
cd src
```
and execute:
```bash
zip src.zip . -rx "*__pycache__/*"
```
This will zip everything within the `src` directory (except for the `__pycache__` directories, if there are any).

You should see a file named `src.zip` appear in the directory.

![src.zip](../assets/kaggle_tutorial/ls_zip.png)

## Uploading The Packaged Code To Kaggle
Now, our strategy is to upload the packaged code as a dataset, so we can add it as to `sys.path` for Python to search for later on.

It should be straight-forward, create a dataset and upload the zip file.

<img src="../assets/kaggle_tutorial/new_notebook.png" width=150>
<img src="../assets/kaggle_tutorial/create_dataset.png" width=306>

You may have also noticed this is when we should upload our training data as a dataset as well. Just follow the same steps as above, zip everything and upload the zip file.

If you intend to use the GTZAN or FMA dataset, then all of the files are already zipped, you just need to upload the zip file.

## Creating A Notebook To Host Your Session
This is where you will assemble everything together. First, create a new notebook. Then, add the datasets you will use for training, as well as the "dataset" leading to the packaged code.

<img src="../assets//kaggle_tutorial/new_notebook.png" width=150>
<img src="../assets//kaggle_tutorial/notebook_view.png" height=347>

In my case, my packaged code is stored in the `genre-classify` dataset. Before moving on, you should also copy the path leading to your packaged code.

It should have the format: `/kaggle/input/your-dataset-name`. For example, my path is `/kaggle/input/genre-classify`.

## Uploading The Training Script To Kaggle
As you may have noticed, we have packaged the code and uploaded the training data. However, we have yet touched the main training script `train.py`.

Once again, we will create another notebook. However, this time, navigate to `File > Editor Type` and select `Script`. You will see the UI change to a script-like editor view.

<img src="../assets/kaggle_tutorial/script_editor.png" height=500>

Now, navigate to `train.py`, copy and paste everything in this file to the Kaggle editor. At the second line in the script, replace the old path with your previous path pointing to the dataset where the packaged code is saved.

![sys_path](../assets/kaggle_tutorial/sys_path.png)

Finally, navigate to `File` item and choose the option `Set as Utility Script`. Now, you can add this as a utility script to your main notebook. You should see the following inputs in your notebook:

![final_input](../assets/kaggle_tutorial/final_input.png)

# Running The Code
If you have set up everything, you can now run the code in this notebook.

You can either:
- Upload this notebook to the notebook you have previously created.
- Create, run a Kaggle session and connect to the running server.

Since the first option is relatively straight-forward, I will show you the second one. Note that you will need VSCode and its Jupyter extension to do this.

I also reccommend going with the esecond option as you can have a notebook to edit locally without the hassle of uploading and saving it on Kaggle.

## How To Connect To The Running Kaggle Session?
Once you have started a session, navigate to `Run` and select `Kaggle Jupyter Server`. You will see a panel pop up on the right hand side.

<img src="../assets/kaggle_tutorial/kaggle_server.png" width=400>
<img src="../assets/kaggle_tutorial/kaggle_server_1.png" height=410>

As you can see, there is a link called `VSCode Compatible URL`. Go ahead and copy this link.

In this notebook, click on the `Select Kernel` option and choose `Select Another Kernel... > Existing Jupyter Server...`. Then you will be prompted to enter a URL to connect to the remote server, enter your copied URL.

![server](../assets/kaggle_tutorial/jupyter_server.png)

After that, proceed as prompted and you will get to choose a kernel to connect to. You are now ready to run the code below. 

#
Remember to copy the path to your training script and paste it into the first line of cell number 5.

For example, my path is `/kaggle/usr/lib/train_genre_classify/genre_classify_train_script.py`.

You may also alter any code as desired.

In [ ]:
# We are uninstalling the datasets library because it clashes with our packages
%pip uninstall -y datasets

# These are the required dependencies you need to install before running
%pip install -qU tensorflow

# This one is optional, you only need this if you intend to use 8-bit optimizers
%pip install -q bitsandbytes

In [2]:
from IPython.display import clear_output

import subprocess, itertools, functools, yaml
from tqdm import tqdm

# Configurations
Change your configurations here, it should have the same structure
as the `train_config.yml` file.

Similarly, specify the model architecture that you want to train in the second
cell of this section. It should have the same structure as the `model.yml` file.

In [ ]:
default_config = {
    "data_args": {
        "type": "fma",
        # "root": "/kaggle/input/gtzan-music",
        "root": [
            "/kaggle/input/fma-small/fma_metadata/fma_metadata",
            "/kaggle/input/fma-small/fma_small/fma_small"
        ],
        "subset_ratio": .8,
        "sampling_rate": 22050,
        "seed": 4,
        "train_ratio": .8,
        "first_n_secs": 20,
        "random_crops": 0,
    },
    "inout": {
        "model_path": "model.yml",
        "logdir": "logs",
        "ckpt_dir": "",
        "checkpoint": None,
    },
    "feature_args": {
        "feature_type": "",
        "n_mels": 128,
        "n_mfcc": 14,
        "n_fft": 2048,
        "window_type": "hann",
    },
    "training_args": {
        "distributed_training": False,
        "mixed_precision": True,
        "epochs": 100,
        "batch_size": 32,
    },
    "optimizer": {
        "type": "adamw",
        "use_8bit_optimizer": False,
        "kwargs": {
            "lr": 1.0e-5,
        }
    },
    "lr_schedulers": {
        "warmup": {
            "total_steps": 20,
            "start_factor": 1e-4
        },
        "decay": {
            "type": 'cosine',
            "kwargs": {'T_max': 80}
        }
    }
}

In [ ]:
models = {
    "cnn_s": {
        "backbone": {
            "type": "cnn",
            "inner_channels": [
                32, 32, 64, 64,
                128, 128, 128,
                256, 256, 256
            ],
            "is_downsampled": [
                0, 1, 0, 1,
                0, 0, 1,
                0, 0, 1
            ],
            "kernel_size": 3,
            "activation": "relu",
            "pooling_type": "max"
        },
        "head": {
            "hidden_dims": [512],
            "dropout_probs": [.2],
            "activation": "relu"
        }
    },
    "cnn_m": {
        "backbone": {
            "type": "cnn",
            "inner_channels": [
                64, 64,
                128, 128, 128,
                256, 256, 256,
                512, 512, 512
            ],
            "is_downsampled": [
                0, 1,
                0, 0, 1,
                0, 0, 1,
                0, 0, 1
            ],
            "kernel_size": 3,
            "activation": "relu",
            "pooling_type": "max"
        },
        "head": {
            "hidden_dims": [1024, 1024],
            "dropout_probs": [.2, .2],
            "activation": "relu"
        }
    },
    "resnet": {
        "backbone": {
            "type": "resnet",
            "inner_channels": [
                128, 128, 128,
                256, 256, 256,
                512, 512, 512,
            ],
            "is_downsampled": [
                0, 0, 0,
                1, 0, 0,
                1, 0, 0,
            ],
            "kernel_size": 3,
            "activation": "relu",
            "pooling_type": "max"
        },
        "head": {
            "hidden_dims": [1024, 1024],
            "dropout_probs": [.2, .2],
            "activation": "relu"
        }
    },
}

# Calling Training Script

In [ ]:
train_script = "/kaggle/usr/lib/train_genre_classify/genre_classify_train_script.py"

def run(
    features, lrs, wcs,
    model_name, ckpt_root, default_config,
    *,
    num_workers=4
):
    pbar = tqdm(
        itertools.product(features, lrs, wcs), desc='Conf',
        total=functools.reduce(
            lambda x, y: x * y, map(len, [features, lrs, wcs])
        )
    )

    for feature, lr, weight_decay in pbar:
        default_config['feature_args']['feature_type'] = feature
        default_config['optimizer']['kwargs']['lr'] = lr
        default_config['optimizer']['kwargs']['weight_decay'] = weight_decay

        weight_decay = f"{weight_decay:.0e}" if weight_decay != 0 else 0
        if feature == 'mel':
            feature += f"-{default_config['feature_args']['n_mels']}"
        elif feature == 'mfcc':
            feature += f"-{default_config['feature_args']['n_mfcc']}"
        default_config['inout']['ckpt_dir'] = (
            f"{ckpt_root}/{feature}/{model_name}_{lr:.0e}_{weight_decay}"
        )
        with open('train_config.yml', 'w', encoding='utf-8') as file:
            yaml.safe_dump(default_config, file, sort_keys=False)

        pbar.set_postfix_str(f"{model_name}, {feature}, {lr=}, {weight_decay=}")
        print()
        subprocess.call(["python", train_script, "-nw", f"{num_workers}", "-d", "cuda:0"])
        clear_output()

In [ ]:
ckpt_root = 'checkpoints'

run_names = ['cnn_s', 'cnn_m']
epochs = {'cnn': 100, 'resnet': 300}
for name, model in models.items():
    if run_names and not name in run_names: continue

    with open('model.yml', 'w', encoding='utf-8') as file:
        yaml.safe_dump(model, file, sort_keys=False)

    epoch = epochs[model['backbone']['type']]
    default_config['training_args']['epochs'] = epoch
    if default_config['lr_schedulers']['decay']['type'] == 'cosine':
        default_config['lr_schedulers']['decay']['kwargs']['T_max'] = (
            epoch - default_config['lr_schedulers']['warmup']['total_steps']
        )

    run(
        ['midi'],
        [1e-5],
        [.01],
        name, 'checkpoints', default_config,
        num_workers=4
    )

You can download all of your checkpoints by zipping it

In [ ]:
!zip -FS -r ckpt_no_pth.zip checkpoints

In [1]:
!rm -r checkpoints